# LangChain, PostgreSQL, Gemini API でドキュメント QA を行う例

以下のパッケージをインストールします。
- LangChain 関係のパッケージ
- PDF の扱いに必要なパッケージ
- CloudSQL の PostgreSQL インスタンスにアクセスするためのパッケージ

In [ ]:
!pip install --user \
  langchain==0.1.0 transformers==4.36.0 \
  pypdf==3.17.0 cryptography==42.0.4 \
  pg8000==1.30.4 cloud-sql-python-connector[pg8000]==1.7.0 \
  langchain-google-vertexai==0.0.6 \
  google-cloud-aiplatform==1.42.1

**注意：次のセルを実行する前にカーネルをリスタートしてください。**

In [ ]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

テキストエンベディング API を試してみます。1つのテキストから、768次元の埋め込みベクトルが得られます。

In [ ]:
from langchain_google_vertexai.embeddings import VertexAIEmbeddings
embeddings = VertexAIEmbeddings(
    model_name='textembedding-gecko-multilingual@001',
    location='asia-northeast1')
embedding_vectors = embeddings.embed_documents(['今日は快晴です。'])

In [ ]:
len(embedding_vectors), len(embedding_vectors[0])

In [ ]:
embedding_vectors[0][:5]

日本人間ドック・予防医療学会 が一般公開している「[検査表の見方](https://www.ningen-dock.jp/public_method/)」をダウンロードします。

In [ ]:
base_url = 'https://raw.githubusercontent.com/gongqi-zhen/HealthBuddy-QA/main'
!wget -q $base_url/PDF/Japan_Society_of_Ningen_Dock_and_Preventive_Medical_Care/shoken-mikata.pdf

PDF ファイルの内容をページごとに分割して、それぞれのページの埋め込みベクトルを生成します。

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
pages = PyPDFLoader('shoken-mikata.pdf').load()
page_contents = [page.page_content for page in pages]
embedding_vectors = embeddings.embed_documents(page_contents)

In [ ]:
len(embedding_vectors), len(embedding_vectors[0])

PostgreSQL に接続するためのコネクションプールを用意します。

In [ ]:
import google.auth
import sqlalchemy
from google.cloud.sql.connector import Connector

_, project_id = google.auth.default()
region = 'asia-northeast1'
instance_name = 'genai-app-db'
INSTANCE_CONNECTION_NAME = '{}:{}:{}'.format(
    project_id, region, instance_name)
DB_USER = 'db-admin'
DB_PASS = 'genai-db-admin'
DB_NAME = 'docs_db'

connector = Connector()

def getconn():
    return connector.connect(
        INSTANCE_CONNECTION_NAME, 'pg8000',
        user=DB_USER, password=DB_PASS, db=DB_NAME)

pool = sqlalchemy.create_engine('postgresql+pg8000://', creator=getconn)

埋め込みベクトルをデータベースから削除、および、データベースに保存する関数を定義します。

In [ ]:
def delete_doc(docid):
    with pool.connect() as db_conn:
        delete_stmt = sqlalchemy.text(
            'DELETE FROM docs_embeddings WHERE docid=:docid;'
        )
        parameters = {'docid': docid}
        db_conn.execute(delete_stmt, parameters=parameters)
        db_conn.commit()

def insert_doc(docid, uid, filename, page, content, embedding_vector):
    with pool.connect() as db_conn:
        insert_stmt = sqlalchemy.text(
            'INSERT INTO docs_embeddings \
             (docid, uid, filename, page, content, embedding) \
             VALUES (:docid, :uid, :filename, :page, :content, :embedding);'
        )
        parameters = {
            'docid': docid,
            'uid': uid,
            'filename': filename,
            'page': page,
            'content': content,
            'embedding': embedding_vector
        }
        db_conn.execute(insert_stmt, parameters=parameters)
        db_conn.commit()

先ほどのドキュメントの各ページの埋め込みベクトルをデータベースに保存します。

In [ ]:
docid = 'dummy_id'
uid = 'dummy_uid'
filename = 'shoken-mikata.pdf'

delete_doc(docid)        
for c, embedding_vector in enumerate(embedding_vectors):
    page = c+1
    insert_doc(docid, uid, filename, page,
               page_contents[c], str(embedding_vector))

質問文から埋め込みベクトルを生成して、関連性の高いページ（埋め込みベクトルの値が近い）のトップ3を取得します。

In [ ]:
question = '血圧値は何を判断する値なのでしょうか。注意するべき値はどのようなものですか？'
question_embedding = embeddings.embed_query(question)

with pool.connect() as db_conn:
    search_stmt = sqlalchemy.text(
        'SELECT filename, page, content, \
                1 - (embedding <=> :question) AS similarity \
         FROM docs_embeddings \
         WHERE uid=:uid \
         ORDER BY similarity DESC LIMIT 3;'
    )
    parameters = {'uid': uid, 'question': str(question_embedding)}
    results = db_conn.execute(search_stmt, parameters=parameters)

text = ''
source = []
for filename, page, content, _ in results:
    source.append({'filename': filename, 'page': page})
    text += content + '\n'

In [ ]:
source

In [ ]:
関連性の高さが一定のしきい値以上の情報だけを使う工夫をしてみる(P.180)

https://mathlandscape.com/cos-similar/

Geminiに聞いた
コサイン類似度は、0から1までの値で、1に近いほど類似度が高いことを示します。閾値の設定は、データセットやタスクによって異なりますが、一般的な目安として以下のような解釈ができます。

閾値の範囲と解釈

0.4 - 0.6:

比較的低い類似度。
関連性はあるかもしれないが、明確な類似とは言えない。
例：複数の文書の中から、キーワードが少し一致する程度。
多くの場合、この範囲の類似度はノイズとみなされることもあります。
0.6 - 0.8:

中程度の類似度。
意味的に類似している可能性が高い。
例：文章のテーマが同じ、または部分的に内容が重複している。
多くのタスクにおいて、この範囲の類似度が適切な閾値となります。
0.8 - 1.0:

高い類似度。
非常に類似していると言える。
例：同じ文章、または内容がほぼ同一の文章。
例えば、文章の重複チェックや剽窃検出など、高い精度が求められるタスクに適しています。


In [17]:
question = '血圧値は何を判断する値なのでしょうか。注意するべき値はどのようなものですか？'
question_embedding = embeddings.embed_query(question)

with pool.connect() as db_conn:
    search_stmt = sqlalchemy.text(
        'SELECT filename, page, content, similarity \
        FROM ( \
            SELECT filename, page, content, \
                1 - (embedding <=> :question) AS similarity \
             FROM docs_embeddings \
             WHERE uid=:uid \
         ) AS subquery \
         WHERE similarity > 0.65 \
         ORDER BY similarity DESC LIMIT 5;'
    )
    parameters = {'uid': uid, 'question': str(question_embedding)}
    results = db_conn.execute(search_stmt, parameters=parameters)

text = ''
source = []
for filename, page, content, similarity in results:
    source.append({'filename': filename, 'page': page, 'similarity': similarity})
    text += content + '\n'

In [18]:
source

[{'filename': 'shoken-mikata.pdf',
  'page': 1,
  'similarity': 0.8093929406968308},
 {'filename': 'shoken-mikata.pdf',
  'page': 2,
  'similarity': 0.7654169229716816},
 {'filename': 'shoken-mikata.pdf',
  'page': 5,
  'similarity': 0.7541800043757546},
 {'filename': 'shoken-mikata.pdf',
  'page': 6,
  'similarity': 0.7506619764577465},
 {'filename': 'shoken-mikata.pdf',
  'page': 7,
  'similarity': 0.7505759817097021}]

得られたページのテキストに基づいて、質問の回答を生成します。

In [19]:
from langchain_google_vertexai import VertexAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain

llm = VertexAI(model_name='gemini-1.5-flash-001', location='asia-northeast1',
               temperature=0.1, max_output_tokens=256)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=6000, chunk_overlap=200)
qa_chain = load_qa_chain(llm, chain_type='refine')
qa_document_chain = AnalyzeDocumentChain(
    combine_docs_chain=qa_chain, text_splitter=text_splitter)

prompt = '{} 日本語で3文程度にまとめて教えてください。'.format(question)
answer = qa_document_chain.invoke({'input_document': text, 'question': prompt})
print(answer['output_text'])

血圧値は心臓のポンプが正常に働いているか、また高血圧かを判断する値です。  収縮期血圧が130～159mmHg、拡張期血圧が85～99mmHgの場合、要注意です。  これらの値は将来、脳・心血管疾患発症しうる可能性を考慮した基準範囲です。 

